In [1]:
import pickle
import os
from bag_of_phrase_words import ChoraleSentences
from bag_of_phrases_utils import *
import torch

chorale_sentences_by_feature_combination_pickle_filename = \
    'chorale_sentences_by_feature_combination.pkl'
bags_of_phrases_directory_name = 'bags_of_phrases'

if chorale_sentences_by_feature_combination_pickle_filename not in os.listdir(os.getcwd()):
    # This is the first time we make the files, let it run.
    feature_combinations = make_feature_combinations()
    make_bags_of_phrases_from_feature_combinations(
        feature_combinations,
        bags_of_phrases_directory_name
    )
    make_chorale_sentences_by_feature_combination(
        chorale_sentences_by_feature_combination_pickle_filename,
        bags_of_phrases_directory_name
    )

with open(chorale_sentences_by_feature_combination_pickle_filename, 'rb') \
        as chorale_sentences_by_feature_combination_file:
    chorale_sentences_by_feature_combination = pickle.load(chorale_sentences_by_feature_combination_file)

basic_chorale_sentences : ChoraleSentences = chorale_sentences_by_feature_combination['basic_two_harmonies']

Processing bags_of_phrases\basic_two_harmonies.pkl
bags_of_phrases\basic_two_harmonies.pkl exists, skipping
Processing bags_of_phrases\basic_three_harmonies_opening.pkl
bags_of_phrases\basic_three_harmonies_opening.pkl exists, skipping
Processing bags_of_phrases\basic_three_harmonies_closing.pkl
bags_of_phrases\basic_three_harmonies_closing.pkl exists, skipping
Processing bags_of_phrases\basic_all_harmonies.pkl
bags_of_phrases\basic_all_harmonies.pkl exists, skipping
Processing bags_of_phrases\mode_two_harmonies.pkl
bags_of_phrases\mode_two_harmonies.pkl exists, skipping
Processing bags_of_phrases\mode_three_harmonies_opening.pkl
bags_of_phrases\mode_three_harmonies_opening.pkl exists, skipping
Processing bags_of_phrases\mode_three_harmonies_closing.pkl
bags_of_phrases\mode_three_harmonies_closing.pkl exists, skipping
Processing bags_of_phrases\mode_all_harmonies.pkl
bags_of_phrases\mode_all_harmonies.pkl exists, skipping


C:\Users\tgewirtz\OneDrive - Intel Corporation\Documents\Technion\SemesterF\ProjectB\BachChoraleGenerator\bag_of_phrase_words.py:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(chorale, dtype=torch.long),
C:\Users\tgewirtz\OneDrive - Intel Corporation\Documents\Technion\SemesterF\ProjectB\BachChoraleGenerator\bag_of_phrase_words.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(chorale, dtype=torch.long) for chorale in self.chorale_sentence_dict.values()


In [2]:
# This is the chorale sentences object which is our processed database
chorale_sentences = basic_chorale_sentences

# For convenient debug, we'll have an option to train on a single example (or just a few) and test on these examples
debug = False

hyperparams = dict(
    h_dim=128,
    n_layers=1,
    dropout=0.1,
    batch_size=1,
    learn_rate=0.1,
    lr_sched_factor=0.9,
    lr_sched_patience=1,
    early_stopping=0
)

In [3]:
from chorale_rnn import *
from trainer import ChoraleSentenceTrainer, ChoraleSentenceTrainerWithFirstPhrase

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

print('hyperparams:\n', hyperparams)

# Dataset definition
vocab_len = chorale_sentences.vocabulary_length

train_test_ratio = 0.9
num_samples = chorale_sentences.number_of_chorales
num_train = int(train_test_ratio * num_samples)

batch_size = min(hyperparams['batch_size'], num_train)

if debug:
    train_index_start = 0
    train_index_end = 1
    test_index_start = 0
    test_index_end = 1
else:
    train_index_start = 0
    train_index_end = num_train
    test_index_start = num_train
    test_index_end = num_samples

# samples, labels = chorale_sentences_tensor_to_labelled_samples(chorale_sentences, device)

samples, labels = chorale_sentences_list_to_labelled_samples(chorale_sentences)

ds_train = list(zip(
    samples[train_index_start:train_index_end], labels[train_index_start:train_index_end]
))
ds_test = list(zip(
    samples[test_index_start:test_index_end], labels[test_index_start:test_index_end]
))

collate_fn = PhraseCollator(chorale_sentences, device)

dl_train = torch.utils.data.DataLoader(ds_train, batch_size, shuffle=False, drop_last=True, collate_fn=collate_fn)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size, shuffle=False, drop_last=True, collate_fn=collate_fn)

print(f'Train: {len(dl_train):3d} batches,')
print(f'Test:  {len(dl_test):3d} batches,')

# Training definition
in_dim = out_dim = vocab_len
checkpoint_file = 'checkpoints/rnn'
num_epochs = 1000
early_stopping = hyperparams['early_stopping']

model = MySequenceModel(
    nn.RNN,
    in_dim,
    hyperparams['h_dim'],
    out_dim,
    hyperparams['n_layers'],
    hyperparams['dropout']
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams['learn_rate'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=hyperparams['lr_sched_factor'],
    patience=hyperparams['lr_sched_patience'],
    verbose=True
)

print(model)

trainer = ChoraleSentenceTrainerWithFirstPhrase(model, loss_fn, optimizer, device)

Using device: cpu
hyperparams:
 {'h_dim': 128, 'n_layers': 1, 'dropout': 0.1, 'batch_size': 1, 'learn_rate': 0.1, 'lr_sched_factor': 0.9, 'lr_sched_patience': 1, 'early_stopping': 0}
Train: 305 batches,
Test:   34 batches,
MySequenceModel(
  (seq_model): RNN(53, 128, batch_first=True, dropout=0.1)
  (out_layer): Linear(in_features=128, out_features=53, bias=True)
)


C:\Users\tgewirtz\Miniconda3\envs\MyChoralesData\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [4]:
def post_epoch_fn(epoch, train_res, test_res, verbose):
    # Update learning rate
    scheduler.step(test_res.accuracy)
    # Sample from model to show progress
    if verbose:
        generated_sequence = generate_from_model(
            model, chorale_sentences, T=0.5
        )
        print(generated_sequence)

# Train, unless final checkpoint is found
checkpoint_file_final = f'{checkpoint_file}_final.pt'
if os.path.isfile(checkpoint_file_final):
    print(f'*** Loading final checkpoint file {checkpoint_file_final} instead of training')
    saved_state = torch.load(checkpoint_file_final, map_location=device)
    model.load_state_dict(saved_state['model_state'])
else:
    try:
        # Print pre-training sampling
        print(generate_from_model(model, chorale_sentences, T=0.5))

        fit_res = trainer.fit(dl_train, dl_test, num_epochs, max_batches=None,
                              post_epoch_fn=post_epoch_fn, early_stopping=early_stopping,
                              checkpoints=checkpoint_file, print_every=1)

    except KeyboardInterrupt as e:
        print('\n *** Training interrupted by user')

[50, 48, 11, 2, 9, 2, 25, 21, 0, 29, 23, 46, 27, 19, 14, 14, 23, 16, 51]
--- EPOCH 1/1000 ---


train_batch:   0%|          | 0/305 [00:00<?, ?it/s]

C:\Users\tgewirtz\OneDrive - Intel Corporation\Documents\Technion\SemesterF\ProjectB\BachChoraleGenerator\bag_of_phrases_utils.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  samples.append(torch.tensor(sample))
C:\Users\tgewirtz\OneDrive - Intel Corporation\Documents\Technion\SemesterF\ProjectB\BachChoraleGenerator\bag_of_phrases_utils.py:138: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels.append(torch.tensor(label))


test_batch:   0%|          | 0/34 [00:00<?, ?it/s]


*** Saved checkpoint checkpoints/rnn.pt
[50, 9, 26, 8, 8, 36, 51]
--- EPOCH 2/1000 ---


...

train_batch:   0%|          | 0/305 [00:00<?, ?it/s]

test_batch:   0%|          | 0/34 [00:00<?, ?it/s]

[50, 0, 4, 51]
--- EPOCH 114/1000 ---


train_batch:   0%|          | 0/305 [00:00<?, ?it/s]


 *** Training interrupted by user


In [10]:
from chorale_rnn import generate_from_model

chorale_sentences.print_chorale_sentence_features(generate_from_model(model, chorale_sentences, T=0.8, pick_first_phrase=True))

Printing features for phrase [5, 4, 7, 32, 15, 0, 4, 4, 1, 51]
5	opening_downbeat_harmony: 5
	fermata_harmony: 4
4	opening_downbeat_harmony: 0
	fermata_harmony: 0
7	opening_downbeat_harmony: 1
	fermata_harmony: 1
32	opening_downbeat_harmony: 0
	fermata_harmony: 2
15	opening_downbeat_harmony: 2
	fermata_harmony: 0
0	opening_downbeat_harmony: 0
	fermata_harmony: 4
4	opening_downbeat_harmony: 0
	fermata_harmony: 0
4	opening_downbeat_harmony: 0
	fermata_harmony: 0
1	opening_downbeat_harmony: 4
	fermata_harmony: 0
